# Loop Transformation

在[之前的教程](https://zhuanlan.zhihu.com/p/553703704)第三章节`Schedules`中我们通过一种基于词典序的方法来表示语句执行的顺序, 这种表示方法称为[Kelly表示](https://ieeexplore.ieee.org/document/472180).但是在`Kelly`表示中，每个语向对应的仿射函数的值域维度并不完全相同. 在比较不同语句实例之间的执行顺序时很不方便. 因此后来有人提出使用[Schedule Tree](https://www.researchgate.net/publication/277075424_Polyhedral_AST_Generation_Is_More_Than_Scanning_Polyhedra),是通过一种树状结构来表示语句的执行顺序的形式. 本篇教程主要介绍如何操作调度树来实现《多面体编译理论与深度学习实践》书中所提到的一些循环变换. 在本文中遇到不理解的概念可以参考[Isl Concept](https://zhuanlan.zhihu.com/p/646406752).

关于调度树的其他文章可以参考[零基础入门isl的调度树-设定篇](https://zhuanlan.zhihu.com/p/641737915), 以及《多面体编译理论与深度学习实践》.

所有教程源码位于[这里](https://github.com/zhen8838/isl_learn).


In [24]:
import pet
import isl


def parse_code(source: str, func_name: str):
  with open("/tmp/parse_code.c", "w") as f:
    f.write(source)
  scop = pet.scop.extract_from_C_source("/tmp/parse_code.c", func_name)
  context = scop.get_context()
  schedule: isl.schedule = scop.get_schedule()
  reads = scop.get_may_reads()
  writes = scop.get_may_writes()
  return (scop, context, schedule, reads, writes)


scop, context, schedule, reads, writes = parse_code("""
void foo()
{
#pragma scop
  const int M = 1024;
  const int N = 2048;
  int a[M*N];
  for (int i = 0; i < M; i++) {
    for (int j = 0; j < N; j++) {                                          
      a[i*N+j] = i*N+j;
    }
  }
#pragma endscop
}
""", "foo")


class CSource():
  def __init__(self, path: str) -> None:
    with open(path, 'r') as f:
      self.context = f.read()

  def _repr_html_(self) -> str:
    return "<pre class='code'><code class=\"cpp hljs\">" + self.context + "</code></pre>"


我们可以dump整个schedule tree, 因为默认是按Flow的方式输出, 所以这里将style修改为block:

In [25]:
def isl_schedule_dump(s: isl.schedule):
  printer = isl.printer.from_file("/tmp/sche.yaml")
  printer = printer.set_yaml_style(isl.ISL_YAML_STYLE.BLOCK)
  printer = printer.print_schedule(s)
  printer.flush()
  with open("/tmp/sche.yaml", 'r') as f:
    content = f.read()
  print(content)

isl_schedule_dump(schedule)

domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_0[{ S_5[i, j] -> [(i)] }]"
      child:
        schedule: "L_1[{ S_5[i, j] -> [(j)] }]"



我们可以直接获取到节点, 或者通过`visitor`进行遍历:

In [26]:
print("直接获取节点:")
print(schedule.get_root().child(0).child(0))


print("后序遍历:")
root_node = schedule.get_root()
def tile_band_node(node: isl.schedule_node):
  print(node.__class__)
  return node
# 使用callback遍历每个节点
_ = root_node.map_descendant_bottom_up(tile_band_node)


直接获取节点:
domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - # YOU ARE HERE
    filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_0[{ S_5[i, j] -> [(i)] }]"
      child:
        schedule: "L_1[{ S_5[i, j] -> [(j)] }]"

后序遍历:
<class 'isl.schedule_node_leaf'>
<class 'isl.schedule_node_filter'>
<class 'isl.schedule_node_leaf'>
<class 'isl.schedule_node_filter'>
<class 'isl.schedule_node_leaf'>
<class 'isl.schedule_node_filter'>
<class 'isl.schedule_node_leaf'>
<class 'isl.schedule_node_filter'>
<class 'isl.schedule_node_leaf'>
<class 'isl.schedule_node_filter'>
<class 'isl.schedule_node_leaf'>
<class 'isl.schedule_node_band'>
<class 'isl.schedule_node_band'>
<class 'isl.schedule_node_filter'>
<class 'isl.schedule_node_sequence'>
<class 'isl.schedule_node_domain'>


## 调度树的操作

在赵捷老师的书中介绍的操作方式比较多:

1. 插入节点, 引入额外信息

2. 修改`band`节点进行循环变换

3. 嵌套的`ban`d节点分裂(`split`)/合并(`combine`), 适配硬件并行维度.

4. 并列的`band`节点(合并)`fuse`/(分块)`distribute`.

虽然在书中第四章中介绍了需要循环变换的方式, 但是缺乏具体的例子来告诉读者怎样将各种变换应用在schedule tree上, 本篇文章主要关注这部分.

首先验证一下输出未经修改的源代码:

In [27]:
from common import CodeGenerator, CSource

generator = CodeGenerator(scop, schedule)
source: CSource = generator.generate()
source


```c
{
  M = 1024;
  N = 2048;
  for (int c0 = 0; c0 <= 1023; c0 += 1)
    for (int c1 = 0; c1 <= 2047; c1 += 1)
      a[2048 * c0 + c1] = (2048 * c0 + c1);
}

```

### 循环反转

`band`节点作为包含循环的节点类型, 需要操作循环也需要通过它所提供的函数. 首先我们可以尝试拿到band节点, 并输出他的`schedule`, 在`schedule node`中, `.schedule`是返回自身的`schedule tree`. 因此band节点的schedule需要使用`get_partial_schedule`来获取`multi_union_pw_aff`.

In [28]:
band_node: isl.schedule_node_band = schedule.get_root().child(0).child(5).child(0)
aff: isl.multi_union_pw_aff = band_node.get_partial_schedule()
print(aff)

L_0[{ S_5[i, j] -> [(i)] }]


这里我想对循环顺序进行反转, 那么我们可以去修改这个`aff`表达式. 正好`isl`中提供了将`aff`函数体进行反转的语法糖:

In [29]:
print("取负操作:", aff.neg())

取负操作: L_0[{ S_5[i, j] -> [(-i)] }]


有了新的`aff`之后, 那么我们需要修改原始的`schedule tree`, 而在`isl`中默认`schedule tree`是不可变的, 所以我们每次插入和构造节点最终都会返回新的`schedule tree`, 因此不需要考虑原始对象的引用问题, 十分方便.

我们只需要删除当前这个循环的band节点, 然后再插入一个由新的`aff`表达式构造的`band`节点即可(由于`band`节点是通过`aff`来构造的, 所以插入的函数为`insert_partial_schedule`):

In [30]:
new_tree = band_node.delete() \
    .insert_partial_schedule(aff.neg()) \
    .schedule()
isl_schedule_dump(new_tree)


domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_0[{ S_5[i, j] -> [(-i)] }]"
      child:
        schedule: "L_1[{ S_5[i, j] -> [(j)] }]"



接下来再生成代码:

In [31]:
generator = CodeGenerator(scop, new_tree)
source: CSource = generator.generate()
source

```c
{
  M = 1024;
  N = 2048;
  for (int c0 = -1023; c0 <= 0; c0 += 1)
    for (int c1 = 0; c1 <= 2047; c1 += 1)
      a[-2048 * c0 + c1] = (-2048 * c0 + c1);
}

```

### 循环交换

我们学会了删除节点和插入节点, 同时也知道了`aff`表达式控制着循环的迭代, 那么想要交换两层循环自然就需要将`schedule tree`上的两个节点位置互换.

首先我们将`child`节点保存下来, 然后从当前节点开始`cut`整个分支之后先插入原本节点,再插入`child`节点:

In [32]:
aff = band_node.get_partial_schedule()
band_node_child: isl.schedule_node_band = band_node.child(0)
child_aff = band_node_child.get_partial_schedule()
new_tree = band_node.cut(). \
  insert_partial_schedule(aff). \
  insert_partial_schedule(child_aff). \
  schedule()

isl_schedule_dump(new_tree)

generator = CodeGenerator(scop, new_tree)
source: CSource = generator.generate()
source

domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_1[{ S_5[i, j] -> [(j)] }]"
      child:
        schedule: "L_0[{ S_5[i, j] -> [(i)] }]"



```c
{
  M = 1024;
  N = 2048;
  for (int c0 = 0; c0 <= 2047; c0 += 1)
    for (int c1 = 0; c1 <= 1023; c1 += 1)
      a[c0 + 2048 * c1] = (c0 + 2048 * c1);
}

```

### 循环延展

循环延展一般是将循环索引变量系数从1改变为一个大于1的整数系数. 其实isl中已经提供了对`band`节点的`aff`表达式操作的接口, 有`scale/scale_down/shift/split/tile`等. 

因此可以使用`scale`去调整循环的系数从而完成循环延展. 在isl中要求我们输入的系数值和这个节点的表达式拥有同样的space, 才可以进行修改, 因此我们需要获取表达式的space, 构造出`multi_val`系数后进行操作:

In [33]:
mv = isl.multi_val.zero(band_node.space())  # 通过aff的space构造multi value, 保证match aff的space.
for i in range(band_node.space().dim(isl.ISL_DIM_TYPE.SET)):
  mv = mv.set_at(i, isl.val("4")) 
print(mv)


{ L_0[4] }


然后调用`scale`函数操作节点,并返回新的`schedule tree`后就可以拿来生成新的代码了, 不过`isl`中应该还有一些特殊处理, 不合理的情况在生成代码时还是被忽略了, 导致`scale`不起作用:

In [34]:
new_tree = band_node.scale(mv).schedule()
isl_schedule_dump(new_tree)
generator = CodeGenerator(scop, new_tree)
source: CSource = generator.generate()
source

domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_0[{ S_5[i, j] -> [(4i)] }]"
      child:
        schedule: "L_1[{ S_5[i, j] -> [(j)] }]"



```c
{
  M = 1024;
  N = 2048;
  for (int c0 = 0; c0 <= 1023; c0 += 1)
    for (int c1 = 0; c1 <= 2047; c1 += 1)
      a[2048 * c0 + c1] = (2048 * c0 + c1);
}

```

不过我们还可以使用`scale_down`, 向下进行延展:

In [35]:
new_tree = band_node.scale_down(mv).schedule()
isl_schedule_dump(new_tree)
generator = CodeGenerator(scop, new_tree)
source: CSource = generator.generate()
source

domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_0[{ S_5[i, j] -> [(floor((i)/4))] }]"
      child:
        schedule: "L_1[{ S_5[i, j] -> [(j)] }]"



```c
{
  M = 1024;
  N = 2048;
  for (int c0 = 0; c0 <= 255; c0 += 1)
    for (int c1 = 0; c1 <= 2047; c1 += 1)
      for (int c2 = 4 * c0; c2 <= 4 * c0 + 3; c2 += 1)
        a[c1 + 2048 * c2] = (c1 + 2048 * c2);
}

```

### 循环分块

分块同样是调用`isl`所提供的接口即可:


In [36]:
mv = isl.multi_val.zero(band_node.space())  # 通过aff的space构造multi value, 保证match aff的space.
for i in range(band_node.space().dim(isl.ISL_DIM_TYPE.SET)):
  mv = mv.set_at(i, isl.val("24")) 

new_tree = band_node.tile(mv).schedule()
isl_schedule_dump(new_tree)
generator = CodeGenerator(scop, new_tree)
source: CSource = generator.generate()
source

domain: "{ S_0[]; S_3[]; S_1[]; S_5[i, j] : 0 <= i <= 1023 and 0 <= j <= 2047; S_2[]; S_4[] }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i, j] }"
    child:
      schedule: "L_0[{ S_5[i, j] -> [(i - (i) mod 24)] }]"
      child:
        schedule: "L_0[{ S_5[i, j] -> [((i) mod 24)] }]"
        child:
          schedule: "L_1[{ S_5[i, j] -> [(j)] }]"



```c
{
  M = 1024;
  N = 2048;
  for (int c0 = 0; c0 <= 1023; c0 += 24)
    for (int c1 = 0; c1 <= min(23, -c0 + 1023); c1 += 1)
      for (int c2 = 0; c2 <= 2047; c2 += 1)
        a[2048 * c0 + 2048 * c1 + c2] = (2048 * c0 + 2048 * c1 + c2);
}

```

### 循环合并

我们给出的这个例子中, 循环`M`和循环`N`实际上是没有依赖的, 所以可以把两个循环合成为一个循环. 我们继续之前修改调度树的思路, 首先我们需要删除一个`band`节点, 并且少了一个循环层级, 为了保证`statement instance`不变, 那么也就需要去修改`domain`节点. 



这里我的做法是首先拿到`domain`的`union set`, 然后修改为新的`domain`.


In [37]:
m = isl.map("{ S_5[i, j] -> S_5[i*2048+j] }")
domain = schedule.get_domain()
new_domain = domain.apply(m)
new_schedule = isl.schedule.from_domain(new_domain)
new_schedule

isl.schedule("""{ domain: "{ S_5[i0] : 0 <= i0 <= 2097151 }" }""")

再按原图顺序重新插入各个`filter`节点.  这里我们知道`sequence`节点下都是`filter`节点, 所以收集各个`filter`节点的信息, 重新构造出`union_set_list`然后统一插入.

In [38]:
old_seq: isl.schedule_node_sequence = schedule.get_root().child(0)
new_seq: isl.schedule_node_sequence = new_schedule.get_root().child(0)

union_set_list = isl.union_set_list(0)
for i in range(old_seq.n_children() - 1):
  old_filter: isl.schedule_node_filter = old_seq.child(i)
  union_set_list = union_set_list.add(old_filter.get_filter())
union_set_list = union_set_list.add(domain.get_space().add_named_tuple("S_5", 1).universe_set())
new_seq = new_seq.insert_sequence(union_set_list)
new_seq

isl.schedule_node_sequence("""domain: "{ S_5[i0] : 0 <= i0 <= 2097151 }"
child:
  # YOU ARE HERE
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i0] }"
""")

接下来再插入`band`节点.

In [39]:
new_band = new_seq.child(5).child(0).insert_partial_schedule(isl.multi_union_pw_aff("L_0[{ S_5[i0] -> [(i0)] }]"))
new_band

isl.schedule_node_band("""domain: "{ S_5[i0] : 0 <= i0 <= 2097151 }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i0] }"
    child:
      # YOU ARE HERE
      schedule: "L_0[{ S_5[i0] -> [(i0)] }]"
""")

最后也就是最关键的一步, 需要将原本代码`stmt`中2维的索引修改为1维的索引操作. 幸好`pet`的代码生成逻辑是根据原始`stmt`构造出一个称为`index`的`multi_pw_aff`, 然后再利用我们当前`schedule`的反向`aff`进行函数合成,也就是将反向`aff`的输出映射到`index`上. 

但是由于原来的`stmt`是2维的索引`S_5[i, j] -> a[(2048i + j)]`, 因此我这里强行把它改成`S_5[i] -> a[i]`, 这样和新的`schedule`中的`aff`进行合成的时候就可以获得正确的`index`. 然后新的index操作取出来变成`ast_expr`, 最后代码生成的时候可以直接使用这个`ast_expr`, 从而生成正确的代码.

In [40]:
new_tree = new_band.schedule()
isl_schedule_dump(new_tree)


def custom_pullback(index: isl.multi_pw_aff, id, iter_map):
  print("old index: ", index)
  print("iter_map: ", iter_map)
  return isl.multi_pw_aff("{ S_5[i] -> a[i] }").pullback(iter_map)


generator = CodeGenerator(scop, new_tree, custom_pullback)
source: CSource = generator.generate()
source


domain: "{ S_5[i0] : 0 <= i0 <= 2097151 }"
child:
  sequence:
  - filter: "{ S_0[] }"
  - filter: "{ S_1[] }"
  - filter: "{ S_2[] }"
  - filter: "{ S_3[] }"
  - filter: "{ S_4[] }"
  - filter: "{ S_5[i0] }"
    child:
      schedule: "L_0[{ S_5[i0] -> [(i0)] }]"

old index:  { S_5[i, j] -> a[(2048i + j)] }
iter_map:  { [[] -> L_0[i0]] -> S_5[(i0)] : 0 <= i0 <= 2097151 }
old index:  { S_5[i, j] -> [(2048i + j)] }
iter_map:  { [[] -> L_0[i0]] -> S_5[(i0)] : 0 <= i0 <= 2097151 }


```c
for (int c0 = 0; c0 <= 2097151; c0 += 1)
  a[c0] = (c0);

```